## LLM as a Judge

## Import models and library

In [1]:
!pip install transformers bitsandbytes sentencepiece accelerate guidance --upgrade -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.4/234.4 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.3 MB/s eta 0:00:00


In [2]:
# for hugging face login to get transformer models from the hub
from huggingface_hub import notebook_login
notebook_login()

In [3]:
!pip install --upgrade transformers -qq
!pip install accelerate

In [4]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import BitsAndBytesConfig
from transformers import pipeline
import pandas as pd


model_name = 'mistralai/Mistral-7B-Instruct-v0.2'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

mistral_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True,
    device_map="auto",
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [6]:
model_name2 = 'stabilityai/StableBeluga-13B'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

gemini_model = AutoModelForCausalLM.from_pretrained(
    model_name2,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True,
    device_map="auto",
)


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [19]:
def get_mistral_answer(prompt):
    """Generates an answer from the Mistral-7B model."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    inputs = tokenizer.encode(prompt, return_tensors='pt', max_length=1024, truncation=True).to(mistral_model.device)
    outputs = mistral_model.generate(inputs, max_length=300, num_return_sequences=1, temperature=0.8, output_scores=True)
    mistral7b_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return mistral7b_response

# def judge_answer(answer, original_prompt):
#     """Evaluates the answer using StableBeluga_13B Model."""
#     tokenizer = AutoTokenizer.from_pretrained(model_name2)
#     evaluation_prompt = f"Please evaluate the following answer to the prompt:\n\nPrompt: {original_prompt}\n\nAnswer: {answer}\n\nProvide your evaluation in a concise and informative manner, focusing on aspects like accuracy, clarity, and relevance."
#     inputs = tokenizer.encode(evaluation_prompt, return_tensors='pt', max_length=1024, truncation=True)
#     outputs = gemini_model.generate(inputs, max_new_tokens=150, num_return_sequences=1, temperature=0.8, output_scores=True)
#     evaluation = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return evaluation
def judge_answer(answer, original_prompt):
    """Evaluates the answer using StableBeluga_13B Model."""
    tokenizer = AutoTokenizer.from_pretrained(model_name2)
    evaluation_prompt = f"""Please evaluate the following answer to the prompt on a scale of 1 to 5 (1 being very poor and 5 being excellent):

    Prompt: {original_prompt}

    Answer: {answer}

    Provide the rating and a brief justification for your score, focusing on aspects like accuracy, clarity, and relevance."""

    inputs = tokenizer.encode(evaluation_prompt, return_tensors='pt', max_length=1024, truncation=True).to(gemini_model.device)
    outputs = gemini_model.generate(inputs, max_new_tokens=150, num_return_sequences=1, temperature=0.8, output_scores=True)
    evaluation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return evaluation


prompt = "What is nuclear fusion?"
mistral_answer = get_mistral_answer(prompt)
judgement = judge_answer(mistral_answer, prompt)

print(f"Mistral's answer: {mistral_answer}")
print(f"StableBeluga_13B evaluation: {judgement}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Mistral's answer: What is nuclear fusion?

Nuclear fusion is a process in which two or more atomic nuclei combine to form a heavier nucleus, releasing energy in the process. This energy is released in the form of light and heat. The sun and other stars generate their energy through nuclear fusion.

In a nuclear fusion reaction, the nuclei must overcome the electrostatic force that keeps them apart due to their positive charges. This requires extremely high temperatures and pressures to bring the nuclei close enough together for the fusion reaction to occur.

The most common type of nuclear fusion reaction is the fusion of hydrogen isotopes, specifically deuterium and tritium, to form helium. This reaction releases a large amount of energy and is the reaction that powers the sun and other stars.

Nuclear fusion has the potential to be a clean and virtually limitless source of energy, as the fuel (hydrogen isotopes) is abundant and the only byproduct is water (if the reaction is controll

In [20]:
def judge_answer(answer, original_prompt):
    """Evaluates the answer using StableBeluga_13B Model."""
    tokenizer = AutoTokenizer.from_pretrained(model_name2)
    evaluation_prompt = f"""Please evaluate the following answer based on these criteria:

    Criteria:
    - Accuracy (1-5):
    - Clarity (1-5):
    - Relevance (1-5):

    Prompt: {original_prompt}

    Answer: {answer}

    Provide a score for each criterion and a brief justification for each score."""

    inputs = tokenizer.encode(evaluation_prompt, return_tensors='pt', max_length=1024, truncation=True).to(gemini_model.device)
    outputs = gemini_model.generate(inputs, max_new_tokens=150, num_return_sequences=1, temperature=0.8, output_scores=True)
    evaluation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return evaluation


prompt = "What is nuclear fusion?"
mistral_answer = get_mistral_answer(prompt)
judgement = judge_answer(mistral_answer, prompt)

print(f"Mistral's answer: {mistral_answer}")
print(f"StableBeluga_13B evaluation: {judgement}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Mistral's answer: What is nuclear fusion?

Nuclear fusion is a process in which two or more atomic nuclei combine to form a heavier nucleus, releasing energy in the process. This energy is released in the form of light and heat. The sun and other stars generate their energy through nuclear fusion.

In a nuclear fusion reaction, the nuclei must overcome the electrostatic force that keeps them apart due to their positive charges. This requires extremely high temperatures and pressures to bring the nuclei close enough together for the fusion reaction to occur.

The most common type of nuclear fusion reaction is the fusion of hydrogen isotopes, specifically deuterium and tritium, to form helium. This reaction releases a large amount of energy and is the reaction that powers the sun and other stars.

Nuclear fusion has the potential to be a clean and virtually limitless source of energy, as the fuel (hydrogen isotopes) is abundant and the only byproduct is water (if the reaction is controll

In [21]:
def get_mistral_answer(prompt):
    """Generates an answer from the Mistral-7B model."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    inputs = tokenizer.encode(prompt, return_tensors='pt', max_length=1024, truncation=True).to(mistral_model.device)
    outputs = mistral_model.generate(inputs, max_length=300, num_return_sequences=1, temperature=0.8, output_scores=True)
    mistral7b_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return mistral7b_response

def judge_answer(answer, original_prompt):
    """Evaluates the answer using StableBeluga_13B Model."""
    tokenizer = AutoTokenizer.from_pretrained(model_name2)
    evaluation_prompt = f"""Please evaluate the following answer to the prompt on a scale of 1 to 5 (1 being very poor and 5 being excellent):

    Prompt: {original_prompt}

    Answer: {answer}

    Provide the rating and a brief justification for your score, focusing on aspects like accuracy, clarity, and relevance."""

    inputs = tokenizer.encode(evaluation_prompt, return_tensors='pt', max_length=1024, truncation=True).to(gemini_model.device)
    outputs = gemini_model.generate(inputs, max_new_tokens=150, num_return_sequences=1, temperature=0.8, output_scores=True)
    evaluation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return evaluation


prompt = "tell me about the current ai situation in the world?"
mistral_answer = get_mistral_answer(prompt)
judgement = judge_answer(mistral_answer, prompt)

print(f"Mistral's answer: {mistral_answer}")
print(f"StableBeluga_13B evaluation: {judgement}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Mistral's answer: tell me about the current ai situation in the world?

The current state of AI (Artificial Intelligence) technology in the world is rapidly advancing and becoming increasingly integrated into various industries and aspects of daily life. Here are some key points about the current AI situation:

1. Research and Development: AI research and development are ongoing in various fields such as computer vision, natural language processing, robotics, and machine learning. Companies like Google, Microsoft, IBM, and Facebook are investing heavily in AI research and development.
2. Applications: AI is being used in various applications such as image and speech recognition, autonomous vehicles, healthcare diagnosis, financial analysis, and customer service. AI is also being used in gaming, entertainment, and education.
3. Ethics and Regulation: As AI becomes more advanced and integrated into society, there are ethical and regulatory concerns. Issues such as bias, privacy, security

In [23]:
def judge_answer(answer, original_prompt):
    """Evaluates the answer using StableBeluga_13B Model."""
    tokenizer = AutoTokenizer.from_pretrained(model_name2)
    evaluation_prompt = f"""Please evaluate the following answer based on these criteria:

    Criteria:
    - Accuracy (1-5):
    - Clarity (1-5):
    - Relevance (1-5):

    Prompt: {original_prompt}

    Answer: {answer}

    Provide a score for each criterion and a brief justification for each score."""

    inputs = tokenizer.encode(evaluation_prompt, return_tensors='pt', max_length=1024, truncation=True).to(gemini_model.device)
    outputs = gemini_model.generate(inputs, max_new_tokens=150, num_return_sequences=1, temperature=0.8, output_scores=True)
    evaluation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return evaluation


prompt = "tell me about the current ai situation in the world?"
mistral_answer = get_mistral_answer(prompt)
judgement = judge_answer(mistral_answer, prompt)

print(f"Mistral's answer: {mistral_answer}")
print(f"StableBeluga_13B evaluation: {judgement}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Mistral's answer: tell me about the current ai situation in the world?

The current state of AI (Artificial Intelligence) technology in the world is rapidly advancing and becoming increasingly integrated into various industries and aspects of daily life. Here are some key points about the current AI situation:

1. Research and Development: AI research and development are ongoing in various fields such as computer vision, natural language processing, robotics, and machine learning. Companies like Google, Microsoft, IBM, and Facebook are investing heavily in AI research and development.
2. Applications: AI is being used in various applications such as image and speech recognition, autonomous vehicles, healthcare diagnosis, financial analysis, and customer service. AI is also being used in gaming, entertainment, and education.
3. Ethics and Regulation: As AI becomes more advanced and integrated into society, there are ethical and regulatory concerns. Issues such as bias, privacy, security